In [1]:
import os
from flask import Flask, request, redirect, url_for, render_template,send_from_directory
import cv2
from ultralytics import YOLO
import io
from PIL import Image
from moviepy.editor import VideoFileClip

In [ ]:
app = Flask(__name__)


app.config['ALLOWED_EXTENSIONS'] = {'mp4', 'avi', 'mov'}
app.config['UPLOAD_FOLDER'] = 'uploads_video'
app.config['OUTPUT_FOLDER'] ='output_video'



model = YOLO('model/best.pt')
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)


def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']



@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    f = request.files['file']
    if f.filename == '':
        return redirect(request.url)
    if f and allowed_file(f.filename):
        filename = f.filename
        print(filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        print(filepath)
        # Le chemin pour accéder au fichier
        f.save(filepath) #Sauvegarder le fichier
        detection(f,filepath)
        single_file_path =retrieve_result()
        clip = VideoFileClip(single_file_path)
        clip.write_videofile("output_video/out.mp4", codec='libx264')
        return redirect(url_for('display_video', filename='out.mp4'))
    return redirect(url_for('index'))

def detection(file,filepath):
    out = None
    file_extension = file.filename.rsplit('.',1)[1].lower()
    if file_extension == 'jpg':
            img = cv2.imread(filepath)
            frame = cv2.imencode('.jpg', cv2.UMat(img))[1].tobytes()
            
            image = Image.open(io.BytesIO(frame))
            
            # Perform detection
            yolo = YOLO('model/best.pt')
            detections = yolo.predict(image, save = True)
    if file_extension =='mp4':
        yolo = YOLO('model/best.pt')
 
 
        detections = yolo.predict(filepath,save= True) 
def retrieve_result():

    folder_path = 'runs\detect'

# Lister les sous-dossiers
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

    if not subfolders:
        raise Exception("Aucun sous-dossier trouvé dans le répertoire spécifié.")

# Trouver le sous-dossier le plus récent
    latest_subfolder = max(subfolders, key=lambda x: os.path.getctime(os.path.join(folder_path, x)))

# Chemin complet du sous-dossier le plus récent
    latest_subfolder_path = os.path.join(folder_path, latest_subfolder)

# Lister les fichiers dans le sous-dossier le plus récent
    files = [f for f in os.listdir(latest_subfolder_path) if os.path.isfile(os.path.join(latest_subfolder_path, f))]

    if len(files) != 1:
        raise Exception("Le sous-dossier ne contient pas exactement un fichier.")

# Chemin complet du fichier unique
    single_file_path = os.path.join(latest_subfolder_path, files[0])

    print(f"Le chemin du fichier est : {single_file_path}")
    return single_file_path

    """folder_path = 'runs/detect'
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    print('les sous dossiers', subfolders)
    latest_subfolder = max(subfolders, key = lambda x: os.path.getctime(os.path.join(folder_path, x)))
    print("le dernier dossier" , latest_subfolder)
    path = folder_path+'/'+latest_subfolder
    print('le chemin ',path)
    print('le contenu du dernier dossier', latest_subfolder[0])
    media_path = os.path.join(path, path[0])
    print('le chemin final', media_path)
   """
@app.route('/display_video/<filename>')
def display_video(filename):
    return render_template('display.html', filename=filename)

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['OUTPUT_FOLDER'], filename)

            



        
if __name__ == '__main__':
    app.run(host='0.0.0.0')



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.114.242:5000
Press CTRL+C to quit


VID-20240608-WA0001.mp4
uploads_video\VID-20240608-WA0001.mp4


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1200) c:\Users\MSI\OneDrive\Desktop\projets_pfe\video_detector_cli\uploads_video\VID-20240608-WA0001.mp4: 384x640 1 Chellah, 460.2ms
video 1/1 (frame 2/1200) c:\Users\MSI\OneDrive\Desktop\projets_pfe\video_detector_cli\uploads_video\VID-20240608-WA0001.mp4: 384x640 1 Chellah, 320.0ms
video 1/1 (frame 3/1200) c:\Users\MSI\OneDrive\Desktop\projets_pfe\video_detector_cli\upload

192.168.114.200 - - [08/Jun/2024 11:32:56] "                    POST /upload HTTP/1.1" 302 -


Moviepy - Done !
Moviepy - video ready output_video/out.mp4


In [3]:
# The right one 


app = Flask(__name__)


app.config['ALLOWED_EXTENSIONS'] = {'mp4', 'avi', 'mov'}
app.config['UPLOAD_FOLDER'] = 'uploads_video'
app.config['OUTPUT_FOLDER'] ='output_video'



model = YOLO('model/best.pt')
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)


def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    f = request.files['file']
    if f.filename == '':
        return redirect(request.url)
    if f and allowed_file(f.filename):
        filename = f.filename
        print(filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        print(filepath)
        # Le chemin pour accéder au fichier
        f.save(filepath) #Sauvegarder le fichier
        detection(f,filepath)
        single_file_path =retrieve_result()
        clip = VideoFileClip(single_file_path)
        clip.write_videofile("output_video/out.mp4", codec='libx264')
                
        
        #shutil.copy2(single_file_path, 'output_video')
        
        
        # Écrire la vidéo en MP4
        video_url = url_for('uploaded_file', filename='out.mp4', _external=True)
        return {'video_url': video_url}, 200
    return 'File not allowed', 400

def detection(file,filepath):
    out = None
    file_extension = file.filename.rsplit('.',1)[1].lower()
    if file_extension == 'jpg':
            img = cv2.imread(filepath)
            frame = cv2.imencode('.jpg', cv2.UMat(img))[1].tobytes()
            
            image = Image.open(io.BytesIO(frame))
            
            # Perform detection
            yolo = YOLO('model/best.pt')
            detections = yolo.predict(image, save = True)
    if file_extension =='mp4':
        yolo = YOLO('model/best.pt')
 
 
        detections = yolo.predict(filepath,save= True) 
def retrieve_result():

    folder_path = 'runs\detect'

# Lister les sous-dossiers
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

    if not subfolders:
        raise Exception("Aucun sous-dossier trouvé dans le répertoire spécifié.")

# Trouver le sous-dossier le plus récent
    latest_subfolder = max(subfolders, key=lambda x: os.path.getctime(os.path.join(folder_path, x)))

# Chemin complet du sous-dossier le plus récent
    latest_subfolder_path = os.path.join(folder_path, latest_subfolder)

# Lister les fichiers dans le sous-dossier le plus récent
    files = [f for f in os.listdir(latest_subfolder_path) if os.path.isfile(os.path.join(latest_subfolder_path, f))]

    if len(files) != 1:
        raise Exception("Le sous-dossier ne contient pas exactement un fichier.")

# Chemin complet du fichier unique
    single_file_path = os.path.join(latest_subfolder_path, files[0])

    print(f"Le chemin du fichier est : {single_file_path}")
    return single_file_path

    """folder_path = 'runs/detect'
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    print('les sous dossiers', subfolders)
    latest_subfolder = max(subfolders, key = lambda x: os.path.getctime(os.path.join(folder_path, x)))
    print("le dernier dossier" , latest_subfolder)
    path = folder_path+'/'+latest_subfolder
    print('le chemin ',path)
    print('le contenu du dernier dossier', latest_subfolder[0])
    media_path = os.path.join(path, path[0])
    print('le chemin final', media_path)
   """


@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['OUTPUT_FOLDER'], filename)

            

if __name__ == '__main__':
    app.run(host='0.0.0.0')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.114.242:5000
Press CTRL+C to quit


VID-20240608-WA0004.mp4
uploads_video\VID-20240608-WA0004.mp4


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/469) c:\Users\MSI\OneDrive\Desktop\projets_pfe\video_detector_cli\uploads_video\VID-20240608-WA0004.mp4: 384x640 1 Chellah, 461.4ms
video 1/1 (frame 2/469) c:\Users\MSI\OneDrive\Desktop\projets_pfe\video_detector_cli\uploads_video\VID-20240608-WA0004.mp4: 384x640 1 Chellah, 350.6ms
video 1/1 (frame 3/469) c:\Users\MSI\OneDrive\Desktop\projets_pfe\video_detector_cli\uploads_v

192.168.114.200 - - [08/Jun/2024 11:38:32] "POST /upload HTTP/1.1" 200 -


Moviepy - Done !
Moviepy - video ready output_video/out.mp4


192.168.114.200 - - [08/Jun/2024 11:38:33] "GET /uploads/out.mp4 HTTP/1.1" 200 -
